In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/IT4772E - NLP/marianmt/

/content/drive/MyDrive/IT4772E - NLP/marianmt


In [ ]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


In [ ]:
def normalize_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.strip().lower()
    text = re.sub(r"\s+", " ", text)  # Remove redundant spaces
    text = re.sub(r"&[^;]+;", "", text)  # Remove HTML entities
    text = re.sub(r"[<>\\\[\]{}~^|]", "", text)  # Remove noisy chars
    return text

In [ ]:
def is_valid_pair(en, vi, min_len=3, max_len=100):
    if not en or not vi:
        return False
    if len(en.split()) < min_len or len(vi.split()) < min_len:
        return False
    if len(en.split()) > max_len or len(vi.split()) > max_len:
        return False
    return True

In [ ]:
def load_split(folder, split_name):
    en_path = os.path.join(folder, f"{split_name}.en")
    vi_path = os.path.join(folder, f"{split_name}.vi")

    with open(en_path, encoding="utf-8") as f:
        en_lines = [line.strip() for line in f]
    with open(vi_path, encoding="utf-8") as f:
        vi_lines = [line.strip() for line in f]

    assert len(en_lines) == len(vi_lines), f"❌ Mismatch in line count for {split_name}"

    data = []
    for en, vi in zip(en_lines, vi_lines):
        en_clean = normalize_text(en)
        vi_clean = normalize_text(vi)
        if is_valid_pair(en_clean, vi_clean):
            data.append({
                "translation": {
                    "en": en_clean,
                    "vi": vi_clean
                }
            })
    return data

In [ ]:
def prepare_phomt(root_folder, output_path="phomt_cleaned.json", combine_all=True):
    print(f"📁 Loading PhoMT from: {root_folder}")
    splits = ["train", "dev", "test"]
    data_all = []

    for split in splits:
        folder_path = os.path.join(root_folder, "detokenization", split)
        print(f"🔹 Processing split: {split}")
        split_data = load_split(folder_path, split)
        print(f"✅ {split}: {len(split_data)} valid pairs")

        if combine_all:
            data_all.extend(split_data)
        else:
            out_file = f"{split}.json" # out_file = f"{output_path.replace('.json', '')}_{split}.json"
            with open(out_file, "w", encoding="utf-8") as f:
                json.dump(split_data, f, ensure_ascii=False, indent=2)
            print(f"💾 Saved: {out_file}")

    if combine_all:
        print(f"\n📦 Total cleaned pairs: {len(data_all)}")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data_all, f, ensure_ascii=False, indent=2)
        print(f"✅ Saved combined file to {output_path}")

In [ ]:
if __name__ == "__main__":
    prepare_phomt(
        root_folder=r"D:\PhoMT",  # ✅ đường dẫn gốc chứa folder "detokenization"
        output_path="phomt_cleaned.json",
        combine_all=True  # Nếu muốn chia nhỏ theo split thì đặt False
    )

In [ ]:
pip install datasets transformers sacrebleu sentencepiece evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00


In [ ]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
import os
import re
import json
import unicodedata
from tqdm import tqdm
import datasets
import evaluate
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
drive_file_path = "/content/drive/MyDrive/IT4772E - NLP/marianmt/phomt_cleaned.json"
local_file_path = "/content/phomt_cleaned.json"
!cp "{drive_file_path}" "{local_file_path}"

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files=local_file_path, split="train")
dataset = dataset.train_test_split(test_size=0.02)  # 98% train, 2% eval

print(dataset["train"][0])
# {'translation': {'en': 'hello world', 'vi': 'xin chào thế giới'}}

Generating train split: 0 examples [00:00, ? examples/s]

{'translation': {'en': 'you were told when you started this was a top secret programme.', 'vi': 'mọi người đã được biết đây là một chương trình tối mật.'}}


In [ ]:
def preprocess(examples):
    sources = [ex["en"] for ex in examples["translation"]]
    targets = [ex["vi"] for ex in examples["translation"]]
    inputs = tokenizer(sources, truncation=True, padding="longest", max_length=64, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, truncation=True, padding="longest", max_length=64, return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=True, batch_size=1000)

Map:   0%|          | 0/2942049 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60042 [00:00<?, ? examples/s]

In [ ]:
# Lấy 500,000 mẫu từ tập train
train_dataset = tokenized_dataset["train"].select(range(500000))
test_dataset = tokenized_dataset["test"].select(range(1000))

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    max_length=128,
    pad_to_multiple_of=8,
    label_pad_token_id=-100,
)

In [ ]:
bleu = evaluate.load("sacrebleu")

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[l if l != -100 else tokenizer.pad_token_id for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_finetuned_vi",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
    optim="adamw_torch",
    warmup_steps=500,
    max_grad_norm=1.0,
    dataloader_num_workers=4,
    report_to="none",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-22-4f90c12f73a6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.494700,0.467140,35.961401
2,0.435700,0.444836,37.268347


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number 

TrainOutput(global_step=11718, training_loss=0.4952930552822308, metrics={'train_runtime': 6278.2704, 'train_samples_per_second': 238.919, 'train_steps_per_second': 1.866, 'total_flos': 2.541889233145037e+16, 'train_loss': 0.4952930552822308, 'epoch': 2.999424})

In [ ]:
# Lưu mô hình
trainer.save_model("./marian_finetuned_vi_final")

In [ ]:
def postprocess(preds, labels):
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return preds, labels

results = trainer.predict(tokenized_dataset["test"])
preds, labels = postprocess(results.predictions, results.label_ids)

bleu_score = bleu.compute(predictions=preds, references=[[l] for l in labels])
print(f"BLEU: {bleu_score['score']:.2f}")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


BLEU: 38.90
